### Question: Interaction and CV in Multiple Linear Regression Models
Consider the `Boston` data set in package `MASS`. This data frame contains the following columns:

- 'crim'  per capita crime rate by town.
- 'zn' proportion of residential land zoned for lots over 25,000 sq.ft.
- 'indus' proportion of non-retail business acres per town.
- 'chas' Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
- 'nox' nitrogen oxides concentration (parts per 10 million).
- 'rm' average number of rooms per dwelling.
- 'age' proportion of owner-occupied units built prior to 1940.
- 'dis' weighted mean of distances to five Boston employment centres.
- 'rad' index of accessibility to radial highways.
- 'tax' full-value property-tax rate per $10,000.
- 'ptratio' pupil-teacher ratio by town.
- 'black' 1000$(Bk-0.63)^2$ where Bk is the proportion of blacks by town.
- 'lstat' lower status of the population (percent).
- 'medv' median value of owner-occupied homes in $1000s.


(1) Use 'medv' as the response variable, fit the  multiple linear regression model $E(medv)=\beta_0+\beta_1 lstat+ \beta_2 rm$.  Are all variables significant?

**Ans:** 

In [1]:
import statsmodels.api as sm
Boston = sm.datasets.get_rdataset("Boston", "MASS").data
print(Boston.columns)

Index(['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax',
       'ptratio', 'black', 'lstat', 'medv'],
      dtype='object')


In [3]:
from statsmodels.formula.api import ols 
model = ols("medv ~ lstat + rm", data=Boston).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.639
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     444.3
Date:                Mon, 21 Apr 2025   Prob (F-statistic):          7.01e-112
Time:                        09:03:23   Log-Likelihood:                -1582.8
No. Observations:                 506   AIC:                             3172.
Df Residuals:                     503   BIC:                             3184.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3583      3.173     -0.428      0.6

lstat and rm are both significant since their p-values are less than 0.05

(2) Add the interaction term $lstat*rm$ to the model in (1), fit the model and interpret the interaction effect if `lstat` is fixed.

**Ans**: 

In [7]:
model2 = ols("medv ~ lstat + rm + lstat*rm", data=Boston).fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.739
Method:                 Least Squares   F-statistic:                     476.9
Date:                Mon, 21 Apr 2025   Prob (F-statistic):          1.75e-146
Time:                        09:24:04   Log-Likelihood:                -1499.2
No. Observations:                 506   AIC:                             3006.
Df Residuals:                     502   BIC:                             3023.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -29.1245      3.342     -8.713      0.0

In higher lstat areas, the effect of higher rm is lower

(3) Consider the interaction terms $lstat*rm, lstat*ptratio, rm*ptratio, lstat*rm*ptratio$, which terms can be added to the model $E(medv)=\beta_0+\beta_1 lstat+ \beta_2 rm + \beta_3 ptratio$ at significance level 0.01?

**Ans**: 

In [8]:
model3 = ols("medv ~ lstat * rm * ptratio", data=Boston).fit()
print(model3.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.791
Model:                            OLS   Adj. R-squared:                  0.788
Method:                 Least Squares   F-statistic:                     268.8
Date:                Mon, 21 Apr 2025   Prob (F-statistic):          1.23e-164
Time:                        09:26:23   Log-Likelihood:                -1444.5
No. Observations:                 506   AIC:                             2905.
Df Residuals:                     498   BIC:                             2939.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         -218.1978     24.402  

All of the interaction terms have P-values lower than 0.01, so they are all significant at the 1% level

(4) Compare the following three models without interaction terms,  $E(medv)=\beta_0+\beta_1 lstat+ \beta_2 rm + \beta_3 ptratio$, $E(medv)=\beta_0+\beta_1 lstat+ \beta_2 rm + \beta_3 ptratio + \beta_4 dis$ and $E(medv)=\beta_0+\beta_1 lstat+ \beta_2 rm + \beta_3 ptratio + \beta_4 dis + \beta_5 nox$.  Use K-fold cross-validation (with k=10) to check which model is the best model for prediction.

**ANS:**  

In [13]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np

# Define features for each model
features_A = ['lstat', 'rm', 'ptratio']
features_B = features_A + ['dis']
features_C = features_B + ['nox']

# Define dictionary of models
models = {
    'Model A': features_A,
    'Model B': features_B,
    'Model C': features_C
}

# Set up cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
mse_scorer = make_scorer(mean_squared_error)

# Evaluate models
results = {}

for name, features in models.items():
    X = Boston[features]
    y = Boston['medv']
    model = LinearRegression()
    neg_mse_scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=kf)
    avg_mse = -np.mean(neg_mse_scores)
    results[name] = avg_mse

# Print results
for name, mse in results.items():
    print(f"{name}: Average MSE = {mse:.3f}")


Model A: Average MSE = 27.733
Model B: Average MSE = 26.772
Model C: Average MSE = 25.284


Model C is the best for prediction